# Import and run embeddings vectorization of iDigBio sample
This is my (Thomas Deatherage) first run at vectorizing a subset of the data from iDigBio to produce embeddings with the [CLIP](https://github.com/openai/CLIP) model.

To run this notebook, you'll need to make sure you have the right deps installed.  If you're using the Jupyter dev container included with this project, you'll need to `$ docker exec -it nfhm_devcontainer-jupyter-1 bash` and then run `$ pip install git+https://github.com/openai/CLIP.git` and `$ pip install pymongo`

## Import dependencies

In [10]:
import torch
import clip
from PIL import Image
from pymongo import MongoClient
import requests
from io import BytesIO


### Loading clip -- takes a few moments

In [13]:
device="cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

In [14]:
def generate_embeddings(image_url, text):
    # Fetch the image from the URL
    response = requests.get(image_url)
    image = Image.open(BytesIO(response.content))
    
    # Preprocess the image
    image = preprocess(image).unsqueeze(0).to(device)
    text = clip.tokenize([text]).to(device)
    
    # Generate embeddings
    with torch.no_grad():
        image_features = model.encode_image(image)
        text_features = model.encode_text(text)
    
    # Normalize the embeddings
    image_features /= image_features.norm(dim=-1, keepdim=True)
    text_features /= text_features.norm(dim=-1, keepdim=True)
    
    return image_features.cpu().numpy(), text_features.cpu().numpy()


## Example Datum from iDigBio

In [16]:
# Example usage with a record from iDigBio
# Text is a concatenation of the `dwc:higherClassification` and `dwc:vernacularName` fields
image_url = "https://images.collections.yale.edu/iiif/2/ypm:e1ce7f3b-fef8-44a3-bacf-60d65c5a478c/full/!1920,1920/0/default.jpg"
text = "brushfoots; butterflies; butterflies and moths; winged insects; insects; hexapods; arthropods; animals; Animalia; Arthropoda; Hexapoda; Insecta; Pterygota; Endopterygota; Lepidoptera; Ditrysia; Papilionoidea; Nymphalidae"
image_embedding, text_embedding = generate_embeddings(image_url, text)

print("Embeddings: ")
print(image_embedding)
print(text_embedding)


Embeddings: 
[[-5.60033917e-02 -3.17959972e-02 -9.05614067e-03  3.22202370e-02
   2.02837791e-02  2.07540113e-02  1.36874625e-02  3.76000106e-02
  -5.74969407e-03  1.62461940e-02  1.99553184e-02 -4.65960279e-02
   9.76790767e-03  3.49398032e-02 -1.56067451e-02  4.73900000e-03
   6.42741248e-02  2.84817368e-02 -1.78808961e-02 -2.59393342e-02
  -2.88059134e-02  3.61038223e-02  1.43290134e-02  1.78895108e-02
  -2.83237919e-02  1.02997152e-03 -1.60472710e-02  1.21921944e-02
   4.68959510e-02 -7.43976934e-03  2.22862586e-02 -1.07567906e-02
   2.08557677e-02 -1.70202516e-02  5.01147509e-02 -5.04499786e-02
   2.71958914e-02 -1.07433824e-02  2.67875586e-02 -1.86665244e-02
  -7.19632506e-02  2.81828791e-02  5.34232445e-02 -5.67098148e-02
   6.00042334e-03 -1.00860946e-01  2.70879529e-02  2.06459742e-02
  -2.37051733e-02  1.98593698e-02  2.49150097e-02  1.01665277e-02
   3.71353216e-02 -7.74086565e-02 -2.25958433e-02 -5.45770919e-04
  -1.85203832e-02  2.95188855e-02 -1.01998141e-02 -5.89239458e-

### Vectorize and store the embeddings

The following code pulls from the raw data collection in mongo, runs the `generate_embeddings` function and then writes those embeddings to another collection.

In [77]:
username = "root"
password = "example"
host = "mongo"
database_name = "NHFM"
raw_data_collection_name = "idigbio"
vectors_collection_name = "idigbio_embedded"
port = 27017

# Create the MongoDB URI and connect to the client
uri = f"mongodb://{username}:{password}@{host}:{port}/?authSource=admin" #{database_name}?authSource=admin

client = MongoClient(uri)

db = client[database_name]

def _fetch_records(collection, limit): # TODO: Add pagination
    collection = db[collection]
    records = collection.find().limit(limit)
    return records

def _store_embeddings(collection_list, dest_collection):
    to_be_inserted = []
    for record in collection_list:
        text = record['data']['dwc:vernacularName'] + record['data']['dwc:higherClassification']
        image_url = record['media'][0]['data']['ac:accessURI']
        embeddings = generate_embeddings(image_url, text)
        record_to_be_inserted = {
            "text": text,
            "image_url": image_url,
            "image_embedding": embeddings[0], # TODO: Use mongo vector support
            "text_embedding": embeddings[1]
        }
        to_be_inserted.append(record_to_be_inserted)

    print('Storing ' + len(to_be_inserted) + ' records')
    db[dest_collection].insert_many(to_be_inserted)


def fetch_records_and_store_embeddings(raw_data_collection_name, vectors_collection_name):
    limit = 10
    _records = _fetch_records(raw_data_collection_name, limit)
    records = list(_records)
    while (len(records) > 0):
        _store_embeddings(records, vectors_collection_name)
        break # <--- Remove to vectorize entire mongo collection


fetch_records_and_store_embeddings(raw_data_collection_name, vectors_collection_name)


storing records


IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



InvalidDocument: cannot encode object: array([[-5.60033917e-02, -3.17959972e-02, -9.05614067e-03,
         3.22202370e-02,  2.02837791e-02,  2.07540113e-02,
         1.36874625e-02,  3.76000106e-02, -5.74969407e-03,
         1.62461940e-02,  1.99553184e-02, -4.65960279e-02,
         9.76790767e-03,  3.49398032e-02, -1.56067451e-02,
         4.73900000e-03,  6.42741248e-02,  2.84817368e-02,
        -1.78808961e-02, -2.59393342e-02, -2.88059134e-02,
         3.61038223e-02,  1.43290134e-02,  1.78895108e-02,
        -2.83237919e-02,  1.02997152e-03, -1.60472710e-02,
         1.21921944e-02,  4.68959510e-02, -7.43976934e-03,
         2.22862586e-02, -1.07567906e-02,  2.08557677e-02,
        -1.70202516e-02,  5.01147509e-02, -5.04499786e-02,
         2.71958914e-02, -1.07433824e-02,  2.67875586e-02,
        -1.86665244e-02, -7.19632506e-02,  2.81828791e-02,
         5.34232445e-02, -5.67098148e-02,  6.00042334e-03,
        -1.00860946e-01,  2.70879529e-02,  2.06459742e-02,
        -2.370517